In [4]:
# OS Libaries
import numpy as np

In [5]:
# Hyperparameters
state_idx = {'A':0, 'B':1, 'C':2}
states = 4
iterations = 50

In [6]:
# Generate data
X = 'AABBBACABBBACAAAAAAAAABBBACAAAAABACAAAAAABBBBACAAAAAAAAAAAABACABACAABBACAAABBBBACAAABACAAAABACAABACAAABBACAAAABBBBACABBACAAAAAABACABACAAABACAABBBACAAAABACABBACA'

In [7]:
class HMM:
    """
    
    alpha (Joint probability): represents the joint probability of observing all of the given data up to time t and the value of z
    --> The forward variable
    --> alpha_t(i) = P(O_1 0_2 ... 0_T, s_t=i | theta) 
    
    beta  (conditional prob) : the probability of all future data from time t+1 until to T given the value of the state_t = i
    --> The backward variable
    --> B_t(i) = P(O_t+1 O_t+2 ... O_T | s_t=i, theta)
    
    gamma (conditional prob) : probability of being in state j at time t, given the oberservation sequence and the model
    --> y_t(i) = P(s_t=i | x_1:T, theta)
    
    """
    def __init__(self, num_latent_states, observations):
        self.num_latent_states      = num_latent_states
        self.T                      = len(observations)
        self.num_observation_states = len(set(observations))
        
        self.transition_prob      = np.zeros([self.num_latent_states, self.num_latent_states])
        self.emission_prob        = np.zeros([self.num_latent_states, self.num_observation_states])
        self.initial_distribution = np.zeros(self.num_latent_states)
        
        self.alpha = np.zeros([self.T, self.num_latent_states]) # alpha_t(i) = P(x_1:t, s_t=i)
        self.beta  = np.zeros([self.T, self.num_latent_states]) # beta_t(i) = P(x_t+1:T | s_t=i)
        self.gamma = np.zeros([self.T, self.num_latent_states]) # gamma_t(i) = P(s_t=i | x_1:T)
        self.xi    = np.zeros([self.T, self.num_latent_states, self.num_latent_states]) # xi_t(i,j) = P(s_t=i, s_t+1=j | x_1:T)
    
    # Initialise model before training it    
    def random_init(self, seed=False):
        if seed:
            np.random.seed(seed)
        
        # initialise transition prob
        for i in range(self.num_latent_states):
            self.transition_prob[i,:] = np.random.uniform(low=0, high=1, size=self.num_latent_states)
            self.transition_prob[i,:] /= np.sum(self.transition_prob[i,:])
        
        # initialise emission_prob
        for i in range(self.num_latent_states):
            self.emission_prob[i,:] = np.random.uniform(low=0, high=1, size=self.num_observation_states)
            self.emission_prob[i,:] /= np.sum(self.emission_prob[i,:])
            
        self.initial_distribution = np.random.uniform(low=0, high=1, size=self.num_latent_states)
        self.initial_distribution /= np.sum(self.initial_distribution)
    
    def Baum_Welch_E_step(self):
        # computes alpha, beta, gamma, xi
        # forward-backward algorithm
        
        # forward pass
        for i in range(self.num_latent_states):
            self.alpha[0, i] = self.initial_distribution[i] * self.emission_prob[i, state_idx[X[0]]]
        for t in range(1, self.T):
            for i in range(self.num_latent_states):
                self.alpha[t, i] = np.sum([self.alpha[t-1, j] * self.transition_prob[j,i] for j in range(self.num_latent_states)]) * self.emission_prob[i, state_idx[X[t]]]
        
        # backward pass
        self.beta[self.T-1, :] = np.ones((1,self.num_latent_states))
        for t in range(self.T-2, -1, -1):
            for i in range(self.num_latent_states):
                self.beta[t, i] = np.sum([self.emission_prob[j, state_idx[X[t+1]]] * self.transition_prob[i, j] * self.beta[t+1, j]for j in range(self.num_latent_states)])
            
        # marginal
        for t in range(self.T):
            for j in range(self.num_latent_states):
                self.gamma[t,j] = self.alpha[t,j] * self.beta[t,j] / np.sum([self.alpha[t,k]*self.beta[t,k] for k in range(self.num_latent_states)])
        
        # xi
        for t in range(self.T-1):
            for i in range(self.num_latent_states):
                for j in range(self.num_latent_states):
                    self.xi[t,i,j] = self.alpha[t,i] * self.transition_prob[i,j] * self.emission_prob[j, state_idx[X[t+1]]]*self.beta[t+1, j]
                    self.xi[t,i,j] /= np.sum([np.sum([self.alpha[t,i] * self.transition_prob[i,j] *self.emission_prob[j, state_idx[X[t+1]]]*self.beta[t+1,j] for j in range(self.num_latent_states)]) for i in range(self.num_latent_states)])
                
    def Baum_Welch_M_step(self):
        # computes pi, transition prob, emission prob
        indicator = lambda x, y: 1 if x == y else 0
        for i in range(self.num_latent_states):
            self.initial_distribution[i] = self.gamma[0,i]
            for j in range(self.num_latent_states):
                self.transition_prob[i,j] = np.sum([self.xi[t, i, j] for t in range(self.T)])/np.sum([self.gamma[t,i] for t in range(self.T)])
            for k in range(self.num_observation_states):
                self.emission_prob[i,k] = np.sum([self.gamma[t,i] * indicator(k, state_idx[X[t]]) for t in range(self.T)]) / np.sum([self.gamma[t,i] for t in range(self.T)])
        
    def show_params(self):
        print(f'    Initial distribution: \n    {self.initial_distribution}')
        print(f'    Transition probabilities: \n    {self.transition_prob}')
        print(f'    Emission probabilities: \n    {self.emission_prob}')
        
    def train(self, num_iter):
        print('Initial parameters:')
        self.show_params()
        for i in range(num_iter):
            self.Baum_Welch_E_step()
            self.Baum_Welch_M_step()
            if (i+1) % 10 == 0:
                print('\n\n')
                print(f'#### Iteration {i+1} complete ####')
                self.show_params()

In [8]:
model = HMM(num_latent_states=states, observations=X)
model.random_init()
model.train(num_iter=iterations)

Initial parameters:
    Initial distribution: 
    [0.44606238 0.03944651 0.30344257 0.21104853]
    Transition probabilities: 
    [[0.23655755 0.13284714 0.39294615 0.23764917]
 [0.17941003 0.01546392 0.43417673 0.37094933]
 [0.29950997 0.36840148 0.21224321 0.11984533]
 [0.17319173 0.43017391 0.30862994 0.08800443]]
    Emission probabilities: 
    [[0.31516067 0.31155022 0.37328911]
 [0.22786054 0.54685481 0.22528466]
 [0.14872422 0.03757545 0.81370033]
 [0.62089274 0.01382932 0.36527793]]



#### Iteration 10 complete ####
    Initial distribution: 
    [0.02790787 0.00420714 0.01218121 0.95570378]
    Transition probabilities: 
    [[0.50255492 0.11151982 0.04814191 0.33486051]
 [0.21231887 0.00481265 0.03066336 0.75164315]
 [0.32984941 0.13248468 0.04954406 0.48667123]
 [0.06725443 0.57078641 0.31027008 0.03730255]]
    Emission probabilities: 
    [[2.90540791e-01 7.08685445e-01 7.73763924e-04]
 [3.54156490e-01 2.51141567e-01 3.94701943e-01]
 [6.18646811e-01 1.55458967e-01 2.25